## Simple seq2seq practice

[**Implement a character-level sequence-to-sequence model**](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)

This is our training model. It leverages three key features of Keras RNNs:

- The `return_state` contructor argument, configuring a RNN layer to return a list where the first entry is the outputs and the next entries are the internal RNN states. This is used to recover the states of the encoder.
- The `inital_state` call argument, specifying the initial state(s) of a RNN. This is used to pass the encoder states to the decoder as initial states.
- The `return_sequences` constructor argument, configuring a RNN to return its full sequence of outputs (instead of just the last output, which the defaults behavior). This is used in the decoder.

In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [4]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
data_path = 'fra-eng/fra.txt'

### Preprocessing

In [5]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [8]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

#### Texts

In [10]:
input_texts[:3]

['Go.', 'Hi.', 'Run!']

In [11]:
target_texts[:3]  # with start and end

['\tVa !\n', '\tSalut !\n', '\tCours\u202f!\n']

In [17]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)  # number of unique chars
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])  # max number of chars
max_decoder_seq_length = max([len(txt) for txt in target_texts])  # including start and end symbol

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [20]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [24]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

print("encoder input shape:", encoder_input_data.shape)
print("decoder input shape:", decoder_input_data.shape)
print("decoder target shape:", decoder_target_data.shape)  # (batch, time_steps, num_features)

encoder input shape: (10000, 16, 69)
decoder input shape: (10000, 59, 93)
decoder target shape: (10000, 59, 93)


In [25]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

### Training
- Encoder is just a commom `LSTM`
- Decoder is a `LSTM` with `return sequences` and `initial state`

In [52]:
state_c.shape

TensorShape([Dimension(None), Dimension(256)])

In [27]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)  # latent dim 256
encoder_outputs, state_h, state_c = encoder(encoder_inputs)  # output, hidden state and cell state
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, 69)     0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None, 93)     0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 333824      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 256),  358400      input_4[0][0]                    
                                                                 lstm_3[0][1]                     
          

In [34]:
decoder_dense.get_weights()[0].shape

(256, 93)

In [36]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=20,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/20
8000/8000 [==============================] - 25s 3ms/step - loss: 0.6080 - val_loss: 0.6796
Epoch 2/20
8000/8000 [==============================] - 24s 3ms/step - loss: 0.5530 - val_loss: 0.6343
Epoch 3/20
8000/8000 [==============================] - 24s 3ms/step - loss: 0.5143 - val_loss: 0.6062
Epoch 4/20
8000/8000 [==============================] - 24s 3ms/step - loss: 0.4848 - val_loss: 0.5765
Epoch 5/20
8000/8000 [==============================] - 24s 3ms/step - loss: 0.4606 - val_loss: 0.5503
Epoch 6/20
8000/8000 [==============================] - 26s 3ms/step - loss: 0.4384 - val_loss: 0.5370
Epoch 7/20
8000/8000 [==============================] - 24s 3ms/step - loss: 0.4196 - val_loss: 0.5263
Epoch 8/20
8000/8000 [==============================] - 24s 3ms/step - loss: 0.4023 - val_loss: 0.5096
Epoch 9/20
8000/8000 [==============================] - 24s 3ms/step - loss: 0.3866 - val_loss: 0.4925
Epoch 10/20
8000/8000 [==

/Users/jeremy.zhang/anaconda3/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


### Predicting
 To decode a test sentence, we will repeatedly:

- Encode the input sentence and retrieve the initial decoder state
- Run one step of the decoder with this initial state and a "start of sequence" token as target. The output will be the next target character.
- Append the target character predicted and repeat.

In short, input English --> (context) + (`\t`) --> (next char + hidden + cell) --> next ... --> till end 

In [46]:
encoder_model = Model(encoder_inputs, encoder_states)  # encoder_input and output hidden state

# the initial state of decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]  # use hidden and cell state to predict the next 

decoder_outputs = decoder_dense(decoder_outputs)  # previous decoder dense
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [47]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [48]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # get the max prediciton as input for next prediction
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [49]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Donnez-la-moi.

-
Input sentence: Hi.
Decoded sentence: Restez allée !

-
Input sentence: Run!
Decoded sentence: Sourriez-vous !

-
Input sentence: Run!
Decoded sentence: Sourriez-vous !

-
Input sentence: Wow!
Decoded sentence: Attends une porte !

-
Input sentence: Fire!
Decoded sentence: Attends une boisse !

-
Input sentence: Help!
Decoded sentence: Donnez-le-moi.

-
Input sentence: Jump.
Decoded sentence: Souvez-le.

-
Input sentence: Stop!
Decoded sentence: Arrête de toirir.

-
Input sentence: Stop!
Decoded sentence: Arrête de toirir.

